In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import requests
from tqdm import tqdm_notebook
import glob
import time

In [3]:
from moviepy.editor import VideoFileClip
from pydub import AudioSegment
from requests.exceptions import ChunkedEncodingError

In [4]:
os.listdir('../')

['.DS_Store',
 'requirements.txt',
 'synonyms',
 'yappy_search',
 '.gitignore',
 '.git',
 'notebooks']

In [57]:
# Считываем данные из таблицы (CSV или Excel, в зависимости от формата)
file_path = '../../yappy_hackaton_2024_400k.csv'  # укажите правильный путь к файлу
df = pd.read_csv(file_path)  # если у вас Excel, используйте pd.read_excel(file_path)

In [59]:
df = df.reset_index()
df = df.rename(columns={'index': 'index orig'})

In [60]:
df

,index orig,link,description
0,0,https://cdn-st.rutubelist.ru/media/b0/e9/ef285...,"#нарезкистримов , #dota2 , #cs2 , #fifa23 , #m..."
1,1,https://cdn-st.rutubelist.ru/media/39/6c/b31bc...,🤫НЕ ВВОДИ ЭТУ КОМАНДУ В РОБЛОКС ! #shorts #rob...
2,2,https://cdn-st.rutubelist.ru/media/e9/e0/b47a9...,"#boobs , #красивыедевушки , #ass"
3,3,https://cdn-st.rutubelist.ru/media/87/43/b11df...,NaN
4,4,https://cdn-st.rutubelist.ru/media/d1/e7/642dc...,NaN
...,...,...,...
399995,399995,https://cdn-st.rutubelist.ru/media/dd/6a/ac296...,#ceramics #handmade #ceramic #clay #ceramicscu...
399996,399996,https://cdn-st.rutubelist.ru/media/aa/d3/4b4c3...,#красивыедевушки #красотки #грудь #boobs #секс...
399997,399997,https://cdn-st.rutubelist.ru/media/73/05/3f80f...,"#лайфхаки , #эксперименты , #roblox , #игрушки..."
399998,399998,https://cdn-st.rutubelist.ru/media/f8/2b/320f7...,#образ #lookbook #показ #неделямоды #капсула #...


In [148]:
df_part = df.iloc[25000:27500]

In [149]:
df_part

,index orig,link,description
25000,25000,https://cdn-st.rutubelist.ru/media/c0/96/e7499...,"#нарезкистримов , #dota2 , #cs2 , #fifa23 , #m..."
25001,25001,https://cdn-st.rutubelist.ru/media/d0/7b/1163e...,#красивыедевушки #грудь #boobs #купальник #би...
25002,25002,https://cdn-st.rutubelist.ru/media/5c/0e/dcdc9...,#гаджеты #технологии #лайфхаки #распаковка
25003,25003,https://cdn-st.rutubelist.ru/media/e6/14/88f8a...,#наука #научное #научпоп #образование
25004,25004,https://cdn-st.rutubelist.ru/media/a6/68/3f7af...,NaN
...,...,...,...
27495,27495,https://cdn-st.rutubelist.ru/media/9a/ce/8b927...,#путешествия #кудапоехать #туризм #море #горы ...
27496,27496,https://cdn-st.rutubelist.ru/media/17/28/901d1...,#fashion #мода #красота #стиль #модныйлук
27497,27497,https://cdn-st.rutubelist.ru/media/a2/12/31aca...,Уже поели шашлыки)
27498,27498,https://cdn-st.rutubelist.ru/media/56/00/4734a...,#образ #lookbook #показ #неделямоды #капсула #...


In [73]:
import sys

sys.path.insert(
    0,
    "/Users/andrey/Documents/hacks/Yappy/git/",
)

In [74]:
from yappy_search.utils import download_video, extract_audio
from yappy_search.audio_models import SongRecognition
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [117]:
dir_w_video:str = '../../data_video/'

In [118]:
os.makedirs(dir_w_video,exist_ok=True)

In [150]:
# Скачиваем все видео из списка
for index, row in tqdm_notebook(df_part.iterrows()):
    video_url = row['link']
    description = row['description']
    video_name = f"{index}{'_'.join(video_url.split('media')[1].split('/'))}"  # можно использовать описание для имени файла
    video_path = os.path.join(dir_w_video,video_name)
    if os.path.isfile(video_path):
        continue
    response = download_video(video_url, video_path)
    print(f"Downloaded {video_name}.")
print("All videos downloaded successfully.")

0it [00:00, ?it/s]

Downloaded 25000_c0_96_e749990c4c079411c97e092dc71c_fhd.mp4.
Downloaded 25001_d0_7b_1163e5ed46bca1d23c1ee28c0e9f_fhd.mp4.
Downloaded 25002_5c_0e_dcdc974c481dbe6552f091085032_fhd.mp4.
Downloaded 25003_e6_14_88f8a46f40aca20c598e8b131715_fhd.mp4.
Downloaded 25004_a6_68_3f7afdbb4f218847c19f95903136_fhd.mp4.
Downloaded 25005_31_4b_a7b6006f45cba7b6d41e4a7c0d2b_fhd.mp4.
Downloaded 25006_0b_42_f5b56924443bab75e771857c5cd2_fhd.mp4.
Downloaded 25007_91_b3_3a3286a64ca8ae26e4e525b50a62_fhd.mp4.
Downloaded 25008_4b_2a_daa6a1bc49ff9daba7349f230994_fhd.mp4.
Downloaded 25009_a9_bd_b956786c4b20b4f3c0dd9dde5677_fhd.mp4.
Downloaded 25010_d0_50_6d0196314fe78d142e66739dd32a_fhd.mp4.
Downloaded 25011_08_53_21ccb3f242a38078e493995f7fb3_fhd.mp4.
Downloaded 25012_13_b1_31a8c5fc4cf283fc61623fa7c9c2_fhd.mp4.
Downloaded 25013_5a_62_45bc80c24ca4a0f4fd191543f5f5_fhd.mp4.
Downloaded 25014_a8_d3_60c22dbb4be7a6feb9745fc70b8b_fhd.mp4.
Downloaded 25015_f7_5d_1d575b1e4ad0ae7f7108e86315f5_fhd.mp4.
Downloaded 25016_fb_34_4

In [151]:
video_name

'27499_63_df_8a34768a4a72ad9f9705f8ae3db4_fhd.mp4'

In [152]:
response.headers

{'Server': 'Angie', 'Date': 'Mon, 10 Jun 2024 07:12:28 GMT', 'Content-Type': 'video/mp4', 'Content-Length': '4983478', 'Connection': 'keep-alive', 'Last-Modified': 'Wed, 13 Sep 2023 09:04:46 GMT', 'x-rgw-object-type': 'Normal', 'ETag': '"ebf7824fff752dda20fae54384309815"', 'x-amz-request-id': 'tx0000069896a7d533873a0-006666a75c-2d28a6db-ritm', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'Accept-Ranges': 'bytes'}

In [153]:
response.history

[]

In [154]:
response.__sizeof__()

24

In [155]:
# Получение списка всех файлов с расширением .mp4 в директории
video_files = glob.glob(os.path.join(dir_w_video, '*.mp4'))
# Сортировка списка файлов по первому значению при split('_')
sorted_video_files = sorted(video_files, key=lambda x: int(os.path.basename(x).split('_')[0]))

# Вывод отсортированного списка файлов
for video_file in sorted_video_files[:5]:
    print(video_file)

../../data_video/25000_c0_96_e749990c4c079411c97e092dc71c_fhd.mp4
../../data_video/25001_d0_7b_1163e5ed46bca1d23c1ee28c0e9f_fhd.mp4
../../data_video/25002_5c_0e_dcdc974c481dbe6552f091085032_fhd.mp4
../../data_video/25003_e6_14_88f8a46f40aca20c598e8b131715_fhd.mp4
../../data_video/25004_a6_68_3f7afdbb4f218847c19f95903136_fhd.mp4


In [156]:
dir_w_screens = '../../scrins/'
# Получение списка всех файлов с расширением .mp4 в директории
screens_files = glob.glob(os.path.join(dir_w_screens, '*.png'))
# Сортировка списка файлов по первому значению при split('_')
# sorted_screens_files = sorted(screens_files, key=lambda x: int(os.path.basename(x).split('_')[0]))

# Вывод отсортированного списка файлов
for screen_file in screens_files[:5]:
    print(screen_file)

../../scrins/Снимок экрана 2024-06-03 в 20.59.20.png
../../scrins/Снимок экрана 2024-06-03 в 20.59.35.png
../../scrins/Снимок экрана 2024-06-03 в 20.59.46.png
../../scrins/Снимок экрана 2024-06-03 в 20.59.53.png


## проверяем pipeline работы

In [126]:
import torch
import asyncio

In [127]:
import json 

In [86]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [87]:
!ls ../../

15. Yappy
15. Yappy.zip
data_audio
data_video
data_video.zip
git
git.zip
notebooks
scrins
yappy_hackaton_2024_1.pq
yappy_hackaton_2024_1501.pq
yappy_hackaton_2024_2001.pq
yappy_hackaton_2024_2501_w_shazam_whisper_small.pq
yappy_hackaton_2024_400k.csv
yappy_hackaton_2024_40k_only_shazam_sub__2501_5005.pq
ЛЦТ_2024_Шаблон_презентации.pptx


In [157]:
df_part

,index orig,link,description
25000,25000,https://cdn-st.rutubelist.ru/media/c0/96/e7499...,"#нарезкистримов , #dota2 , #cs2 , #fifa23 , #m..."
25001,25001,https://cdn-st.rutubelist.ru/media/d0/7b/1163e...,#красивыедевушки #грудь #boobs #купальник #би...
25002,25002,https://cdn-st.rutubelist.ru/media/5c/0e/dcdc9...,#гаджеты #технологии #лайфхаки #распаковка
25003,25003,https://cdn-st.rutubelist.ru/media/e6/14/88f8a...,#наука #научное #научпоп #образование
25004,25004,https://cdn-st.rutubelist.ru/media/a6/68/3f7af...,NaN
...,...,...,...
27495,27495,https://cdn-st.rutubelist.ru/media/9a/ce/8b927...,#путешествия #кудапоехать #туризм #море #горы ...
27496,27496,https://cdn-st.rutubelist.ru/media/17/28/901d1...,#fashion #мода #красота #стиль #модныйлук
27497,27497,https://cdn-st.rutubelist.ru/media/a2/12/31aca...,Уже поели шашлыки)
27498,27498,https://cdn-st.rutubelist.ru/media/56/00/4734a...,#образ #lookbook #показ #неделямоды #капсула #...


### Делаем только shazam

In [129]:
song_recognition = SongRecognition()

In [130]:
# Функция для повторных попыток скачивания видео
def robust_download_video(video_url, video_path, max_retries=3):
    retries = 0
    while retries < max_retries:
        try:
            response = download_video(video_url, video_path)
            return response
        except ChunkedEncodingError as e:
            print(f"Error downloading {video_url}: {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying... ({retries}/{max_retries})")
            else:
                print(f"Failed to download {video_url} after {max_retries} attempts")
                return None
# Функция для извлечения аудио с проверкой
def extract_audio_with_check(video_path, output_dir):
    try:
        video_clip = VideoFileClip(video_path)
        audio_file_path = os.path.join(
            output_dir, os.path.basename(video_path).replace(".mp4", ".mp3")
        )
        if video_clip.audio is None:
            print(f"No audio found in {video_path}. Creating empty audio file.")
            # Создаем пустой аудиофайл
            silent_audio = AudioSegment.silent(duration=1000)  # 1 секунда тишины
            silent_audio.export(audio_file_path, format="mp3")
        else:
            video_clip.audio.write_audiofile(audio_file_path, verbose=False, logger=None)
        return audio_file_path
    except Exception as e:
        print(f"Failed to extract audio from {video_path}: {e}")
        return None

# Ограничение запросов к Shazam в минуту
async def recognize_audio_with_rate_limiting(audio_path, rate_limit=60):
    await asyncio.sleep(60 / rate_limit)  # Задержка для ограничения запросов
    try:
        return await song_recognition.recognize_audio(audio_path)
    except ClientError as e:
        print(f"Request failed for {audio_path}: {e}")
        return 'ERROR!'

# Обертка для выполнения задачи с тайм-аутом
async def recognize_audio_with_timeout(audio_path, timeout=60):
    try:
        return await asyncio.wait_for(recognize_audio_with_rate_limiting(audio_path), timeout)
    except AsyncioTimeoutError:
        print(f"Recognition for {audio_path} timed out.")
        return 'ERROR!'

In [158]:
# Получение списка всех файлов с расширением .mp4 в директории
video_files = glob.glob(os.path.join(dir_w_video, '*.mp4'))
# Сортировка списка файлов по первому значению при split('_')
sorted_video_files = sorted(video_files, key=lambda x: int(os.path.basename(x).split('_')[0]))

In [159]:
sorted_video_files

['../../data_video/25000_c0_96_e749990c4c079411c97e092dc71c_fhd.mp4',
 '../../data_video/25001_d0_7b_1163e5ed46bca1d23c1ee28c0e9f_fhd.mp4',
 '../../data_video/25002_5c_0e_dcdc974c481dbe6552f091085032_fhd.mp4',
 '../../data_video/25003_e6_14_88f8a46f40aca20c598e8b131715_fhd.mp4',
 '../../data_video/25004_a6_68_3f7afdbb4f218847c19f95903136_fhd.mp4',
 '../../data_video/25005_31_4b_a7b6006f45cba7b6d41e4a7c0d2b_fhd.mp4',
 '../../data_video/25006_0b_42_f5b56924443bab75e771857c5cd2_fhd.mp4',
 '../../data_video/25007_91_b3_3a3286a64ca8ae26e4e525b50a62_fhd.mp4',
 '../../data_video/25008_4b_2a_daa6a1bc49ff9daba7349f230994_fhd.mp4',
 '../../data_video/25009_a9_bd_b956786c4b20b4f3c0dd9dde5677_fhd.mp4',
 '../../data_video/25010_d0_50_6d0196314fe78d142e66739dd32a_fhd.mp4',
 '../../data_video/25011_08_53_21ccb3f242a38078e493995f7fb3_fhd.mp4',
 '../../data_video/25012_13_b1_31a8c5fc4cf283fc61623fa7c9c2_fhd.mp4',
 '../../data_video/25013_5a_62_45bc80c24ca4a0f4fd191543f5f5_fhd.mp4',
 '../../data_video/2

In [160]:
# переводим из видео в аудио
audio_output_dir = os.path.join(dir_w_video.rsplit('/', 2)[0], f'data_audio')
os.makedirs(audio_output_dir, exist_ok=True)
audio_paths = []
for video_path in tqdm_notebook(sorted_video_files):
    audio_file_path = os.path.join(
        audio_output_dir, os.path.basename(video_path).replace(".mp4", ".mp3")
    )
    if os.path.isfile(audio_file_path):
        pass
    else:
        audio_file_path = extract_audio_with_check(video_path, audio_output_dir)
    audio_paths.append(audio_file_path)
print("All audios downloaded successfully.")

  0%|          | 0/2500 [00:00<?, ?it/s]

No audio found in ../../data_video/25268_28_76_c0c5ce6542eeb3bed51fe6c0da5b_fhd.mp4. Creating empty audio file.
No audio found in ../../data_video/25376_d5_dd_f8ea360f4a71ad62083796eb852c_fhd.mp4. Creating empty audio file.
No audio found in ../../data_video/25529_0f_37_eba476424e9fa682fa812b412b65_fhd.mp4. Creating empty audio file.
No audio found in ../../data_video/25559_c1_09_f1962290466fb44a3baeba84d0c6_fhd.mp4. Creating empty audio file.
No audio found in ../../data_video/25584_77_0d_af0a36c9471b931d2e685190e452_fhd.mp4. Creating empty audio file.
No audio found in ../../data_video/25852_62_7d_4532b6304b29bf2bd1a611c65ffa_fhd.mp4. Creating empty audio file.
No audio found in ../../data_video/26102_2e_81_54ffc7f0436a8cf071ed33cf41d5_fhd.mp4. Creating empty audio file.
No audio found in ../../data_video/26225_c1_43_54dfd4dd411cb29d61a00e1268a2_fhd.mp4. Creating empty audio file.
No audio found in ../../data_video/26333_ec_18_ba01cd9b4d528fc068c510020ab5_fhd.mp4. Creating empty audi

In [161]:
assert len(sorted_video_files) == len(audio_paths)

In [162]:
audio_paths_interest = audio_paths

In [163]:
audio_paths_interest[:5],audio_paths_interest[-5:]

(['../../data_audio/25000_c0_96_e749990c4c079411c97e092dc71c_fhd.mp3',
  '../../data_audio/25001_d0_7b_1163e5ed46bca1d23c1ee28c0e9f_fhd.mp3',
  '../../data_audio/25002_5c_0e_dcdc974c481dbe6552f091085032_fhd.mp3',
  '../../data_audio/25003_e6_14_88f8a46f40aca20c598e8b131715_fhd.mp3',
  '../../data_audio/25004_a6_68_3f7afdbb4f218847c19f95903136_fhd.mp3'],
 ['../../data_audio/27495_9a_ce_8b9279b44bed969b7c1bbfef97fb_fhd.mp3',
  '../../data_audio/27496_17_28_901d149b48d89304ea5f184aeacf_fhd.mp3',
  '../../data_audio/27497_a2_12_31acaabd4661ae62ad75081fbba3_fhd.mp3',
  '../../data_audio/27498_56_00_4734a56f443dbba52a0753b87c9e_fhd.mp3',
  '../../data_audio/27499_63_df_8a34768a4a72ad9f9705f8ae3db4_fhd.mp3'])

In [164]:
df_part

,index orig,link,description
25000,25000,https://cdn-st.rutubelist.ru/media/c0/96/e7499...,"#нарезкистримов , #dota2 , #cs2 , #fifa23 , #m..."
25001,25001,https://cdn-st.rutubelist.ru/media/d0/7b/1163e...,#красивыедевушки #грудь #boobs #купальник #би...
25002,25002,https://cdn-st.rutubelist.ru/media/5c/0e/dcdc9...,#гаджеты #технологии #лайфхаки #распаковка
25003,25003,https://cdn-st.rutubelist.ru/media/e6/14/88f8a...,#наука #научное #научпоп #образование
25004,25004,https://cdn-st.rutubelist.ru/media/a6/68/3f7af...,NaN
...,...,...,...
27495,27495,https://cdn-st.rutubelist.ru/media/9a/ce/8b927...,#путешествия #кудапоехать #туризм #море #горы ...
27496,27496,https://cdn-st.rutubelist.ru/media/17/28/901d1...,#fashion #мода #красота #стиль #модныйлук
27497,27497,https://cdn-st.rutubelist.ru/media/a2/12/31aca...,Уже поели шашлыки)
27498,27498,https://cdn-st.rutubelist.ru/media/56/00/4734a...,#образ #lookbook #показ #неделямоды #капсула #...


In [165]:
recognitions_all = []
title_shazam_all = []
subtitle_shazam_all = []
url_shazam_all = []
cnt = 0 

In [166]:
len(title_shazam_all)

0

In [167]:
for audio_path in tqdm_notebook(audio_paths_interest[len(title_shazam_all):]):
    if cnt % 2 == 0:
        time.sleep(2)
    try:    
        loop = asyncio.get_event_loop()
        recognition = loop.run_until_complete(recognize_audio_with_timeout(audio_path))
    except:
        recognition = 'ERROR!'
        print(f"{audio_path}| recognition ERROR")

    if type(recognition) == str:
        title_shazam = ''
        subtitle_shazam = ''
        url_shazam = ''
    else:
        title_shazam = recognition['title']
        subtitle_shazam = recognition['subtitle']
        url_shazam = recognition['url']

    title_shazam_all.append(title_shazam)
    subtitle_shazam_all.append(subtitle_shazam)
    url_shazam_all.append(url_shazam)
    cnt+=1

  0%|          | 0/2500 [00:00<?, ?it/s]

../../data_audio/27407_b4_b2_dee2401340dbb40ade8a10bff8f0_fhd.mp3| recognition ERROR


In [168]:
df_part['title shazam'] = title_shazam_all
df_part['subtitle shazam'] = subtitle_shazam_all
df_part['url shazam'] = url_shazam_all

In [169]:
1+1

2

In [170]:
df_part

,index orig,link,description,title shazam,subtitle shazam,url shazam
25000,25000,https://cdn-st.rutubelist.ru/media/c0/96/e7499...,"#нарезкистримов , #dota2 , #cs2 , #fifa23 , #m...",,,
25001,25001,https://cdn-st.rutubelist.ru/media/d0/7b/1163e...,#красивыедевушки #грудь #boobs #купальник #би...,You Know You Like It,DJ Snake & AlunaGeorge,https://www.shazam.com/track/92279003/you-know...
25002,25002,https://cdn-st.rutubelist.ru/media/5c/0e/dcdc9...,#гаджеты #технологии #лайфхаки #распаковка,,,
25003,25003,https://cdn-st.rutubelist.ru/media/e6/14/88f8a...,#наука #научное #научпоп #образование,,,
25004,25004,https://cdn-st.rutubelist.ru/media/a6/68/3f7af...,NaN,Стрелы,Markul & Тося Чайкина,https://www.shazam.com/track/624549785/%D1%81%...
...,...,...,...,...,...,...
27495,27495,https://cdn-st.rutubelist.ru/media/9a/ce/8b927...,#путешествия #кудапоехать #туризм #море #горы ...,,,
27496,27496,https://cdn-st.rutubelist.ru/media/17/28/901d1...,#fashion #мода #красота #стиль #модныйлук,Give It To Me (Darkroom Edit),Nelly Furtado,https://www.shazam.com/track/545044090/give-it...
27497,27497,https://cdn-st.rutubelist.ru/media/a2/12/31aca...,Уже поели шашлыки),,,
27498,27498,https://cdn-st.rutubelist.ru/media/56/00/4734a...,#образ #lookbook #показ #неделямоды #капсула #...,,,


In [171]:
index_1 = df_part['index orig'].iloc[0]
index_2 = df_part['index orig'].iloc[-1]

In [172]:
save_sub_path = f'../../yappy_hackaton_2024_40k_only_shazam_sub__{index_1}_{index_2}.pq'
df_part.to_parquet(save_sub_path,index=False)

In [147]:
# Очищение файлов после обработки
for video_path in sorted_video_files:
    try:
        os.remove(video_path)
    except:
        pass
for audio_path in audio_paths:
    try:
        if audio_path:
            os.remove(audio_path)
    except:
        pass

### старая проба на cpu whishper small & shazam

In [20]:
from yappy_search.config import ConfigImageCaptioning
from yappy_search.model import VideoProcessor

In [21]:
config = ConfigImageCaptioning()

In [22]:
config

ConfigImageCaptioning(min_length=0, max_length=100, num_beams=4, model_name_image_caption='GeorgeBredis/ruIdefics2-ruLLaVA-merged', model_name_audio_whisper='small', model_name_audio_lang='ru')

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [24]:
device

device(type='cpu')

In [25]:
model_all = VideoProcessor(config,device=device, w_image_caption=False)

In [26]:
video_path = sorted_video_files[0]

In [27]:
audio_output_dir:str|os.PathLike = os.path.join(video_path.rsplit('/',2)[0],'data_audio')
os.makedirs(audio_output_dir, exist_ok=True)

In [ ]:
transcriptions = []
recognitions = []
for num, video_path in tqdm_notebook(enumerate(sorted_video_files)):
    try:
        audio_path = model_all.audio_transcription.extract_audio(video_path, audio_output_dir)
        transcription = model_all.audio_transcription.transcribe(audio_path)
    except:
        transcription = 'ERROR!'
        print(f"{num}| transcription ERROR")
              
    try:    
        loop = asyncio.get_event_loop()
        recognition = loop.run_until_complete(model_all.song_recognition.recognize_audio(audio_path))
    except:
        recognition = 'ERROR!'
        print(f"{num}| recognition ERROR")

    # для записи
    if type(recognition) == dict:
        recognition = json.dumps(recognition)
              
    transcriptions.append(transcription)
    recognitions.append(recognition)

    if num % 500 == 0:
        df_part = df.iloc[:len(transcriptions)].copy(deep=True)
        df_part['transcription'] = transcriptions
        df_part['recognition'] = recognitions
        df_part.to_parquet(f'../../yappy_hackaton_2024_{df_part.shape[0]}.pq', index=False)

0it [00:00, ?it/s]

131| transcription ERROR
546| transcription ERROR
762| transcription ERROR
981| transcription ERROR
1272| transcription ERROR
1324| transcription ERROR
1410| transcription ERROR
1622| transcription ERROR
1719| transcription ERROR
1784| transcription ERROR
1950| transcription ERROR
1974| transcription ERROR
2105| transcription ERROR
2197| transcription ERROR
2285| transcription ERROR
2773| transcription ERROR
2871| transcription ERROR
2871| recognition ERROR
2872| transcription ERROR
2872| recognition ERROR


Task exception was never retrieved
future: <Task finished name='Task-8614' coro=<SongRecognition.recognize_audio() done, defined at /Users/andrey/Documents/hacks/Yappy/git/yappy_search/audio_models.py:31> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/var/folders/yr/mmqsh_3x4z92c40wn0j7x2yw0000gp/T/ipykernel_26771/3650306262.py", line 13, in <module>
    recognition = loop.run_until_complete(model_all.song_recognition.recognize_audio(audio_path))
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/yappy_311/lib/python3.11/site-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/opt/anaconda3/envs/yappy_311/lib/python3.11/site-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/opt/anaconda3/envs/yappy_311/lib/python3.11/asyncio/events.py", line 84, in _run
    self._context.run(self._callback, *self._args)
  File "/opt/anacon

2873| transcription ERROR
2874| transcription ERROR


In [ ]:
1+1

In [ ]:
for index in range(len(transcriptions)):
    row = df.iloc[index]
    print(index)
    print(f'link:{row.link}')
    print(f'description:{row.description}')
    print(f'transcription:{transcriptions[index]}')
    print(f'recognition:{recognitions[index]}')
    print(20*'-')

In [ ]:
# !pip install -U transformers

In [ ]:
# !pip install av bitsandbytes moviepy openai-whisper

In [ ]:
# !pip install accelerate

In [ ]:
import numpy as np
import av
# from peft import LoraConfig
from transformers import AutoProcessor, BitsAndBytesConfig, Idefics2ForConditionalGeneration,AutoModelForVision2Seq
from transformers.image_utils import load_image
import torch
from PIL import Image

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

#### Проба [video LLaVa](https://huggingface.co/spaces/LanguageBind/Video-LLaVA)
[git](https://github.com/PKU-YuanGroup/Video-LLaVA)

In [ ]:
# !pip install -U transformers

In [ ]:
from transformers import VideoLlavaProcessor, VideoLlavaForConditionalGeneration

In [ ]:
def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
model = VideoLlavaForConditionalGeneration.from_pretrained(
    "LanguageBind/Video-LLaVA-7B-hf",
     torch_dtype=torch.float16,
     quantization_config=quantization_config,
)
processor = VideoLlavaProcessor.from_pretrained("LanguageBind/Video-LLaVA-7B-hf")

RuntimeError: No GPU found. A GPU is needed for quantization.

In [ ]:
video_captions = []
for video_path in tqdm_notebook(sorted_video_files[:20]):
    prompt = "USER: <video>Make a summary of video. ASSISTANT:"
    container = av.open(video_path)

    # sample uniformly 8 frames from the video
    total_frames = container.streams.video[0].frames
    indices = np.arange(0, total_frames, total_frames / 8).astype(int)
    # indices = [int(0.25 * total_frames), int(0.5 * total_frames), int(0.75 * total_frames)]
    clip = read_video_pyav(container, indices)
    inputs = processor(text=prompt, videos=clip, return_tensors="pt")

    # Generate
    generate_ids = model.generate(**inputs, max_length=80)
    caption = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    print(caption)
    break

/var/folders/yr/mmqsh_3x4z92c40wn0j7x2yw0000gp/T/ipykernel_39766/235269379.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for video_path in tqdm_notebook(sorted_video_files[:20]):


  0%|          | 0/20 [00:00<?, ?it/s]

NameError: name 'processor' is not defined

In [ ]:
model_name = "GuardisAI/Video-LLaVA-7B-GPTQ-4bit-V1"
#You are using a model of type llava to instantiate a model of type video_llava. This is not supported for all configurations of models and can yield errors.

### [ruIdefics2-ruLLaVA-merged](https://huggingface.co/GeorgeBredis/ruIdefics2-ruLLaVA-merged)

In [ ]:
DEVICE  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
processor = AutoProcessor.from_pretrained(
    "GeorgeBredis/ruIdefics2-ruLLaVA-merged",
    do_image_splitting=False
)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

model = Idefics2ForConditionalGeneration.from_pretrained(
     "GeorgeBredis/ruIdefics2-ruLLaVA-merged",
     torch_dtype=torch.float16,
     quantization_config=quantization_config,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Что изображено на данной картинке?"},
        ]
    }
]

In [ ]:
video_captions = []
for video_path in tqdm_notebook(sorted_video_files):
    # load video
    container = av.open(video_path)

    # get fps (corrected approach)
    fps = container.streams.video[0].average_rate.numerator / container.streams.video[0].average_rate.denominator

    # extract frames at 25%, 50%, & 75%
    seg_len = container.streams.video[0].frames  # total number of frames
    indices = [int(0.25 * seg_len), int(0.5 * seg_len), int(0.75 * seg_len)]
    frames = []

    container.seek(0)
    for i, frame in enumerate(container.decode(video=0)):
        if i in indices:
          frames.append(frame.to_ndarray(format="rgb24"))
    # generate caption
    gen_kwargs = {
        "min_length": 0,
        "max_length": 100,
        "num_beams": 4,
    }

    # Convert numpy arrays to PIL Images
    pil_images = [Image.fromarray(frame) for frame in frames]

    prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(text=prompt, images=pil_images[:1], return_tensors="pt")
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
    generated_ids = model.generate(**inputs, max_new_tokens=100)
    generated_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)
    # caption = '|'.join([pred.strip() for pred in preds])
    caption = generated_texts
    video_captions.append(caption)

<ipython-input-27-5f37e01cd0ee>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for video_path in tqdm_notebook(sorted_video_files):


  0%|          | 0/27 [00:00<?, ?it/s]

InvalidDataError: [Errno 1094995529] Invalid data found when processing input

In [ ]:
for index in range(len(video_captions)):
    row = df.iloc[index]
    print(f'index: {index}|link:{row.link}')
    print(f'description:{row.description}')
    print(f'video_caption:{video_captions[index][0]}')
    print(20*'-')

index: 0|link:https://cdn-st.rutubelist.ru/media/b0/e9/ef285e0241139fc611318ed33071/fhd.mp4
description:#нарезкистримов , #dota2 , #cs2 , #fifa23 , #minecraft , #майнкрафт , #геншин , #genshin
video_caption:User: Что изображено на данной картинке? 
Assistant: На изображении представлен футбольный матч, где группа людей играет в футбол на поле. Они разбросаны по всему полю, некоторые из них стоят ближе к центру, а другие - ближе к краям.

Всего на поле шесть человек, которые активно участвуют в игре. Один из игроков находится в центре по
--------------------
index: 1|link:https://cdn-st.rutubelist.ru/media/39/6c/b31bc6864bef9d8a96814f1822ca/fhd.mp4
description:🤫НЕ ВВОДИ ЭТУ КОМАНДУ В РОБЛОКС ! #shorts #roblox #роблокс
video_caption:User: Что изображено на данной картинке? 
Assistant: На изображении представлен игровой персонаж, стоящий на улице, который, похоже, является частью игры. Он одет в черно-белую форму и, похоже, является игровым персонажем.
--------------------
index: 2|link:h

### [RUDOLPH](https://habr.com/ru/companies/sberbank/articles/733470/)
RuCLIP старее

### [Chat-UniVi-7B-v1.5](https://huggingface.co/Chat-UniVi/Chat-UniVi-7B-v1.5/tree/main)

### [blip-2](https://huggingface.co/blog/blip-2)
[blip2-opt-2.7b](https://huggingface.co/Salesforce/blip2-opt-2.7b)

### [pix2struct](https://huggingface.co/google/pix2struct-textcaps-base)

### проба [SpaceTimeGPT - A Spatiotemporal Video Captioning Model](https://huggingface.co/Neleac/SpaceTimeGPT#spacetimegpt---a-spatiotemporal-video-captioning-model)

In [ ]:
import av
import numpy as np
import torch
from transformers import AutoImageProcessor, AutoTokenizer, VisionEncoderDecoderModel

device = "cuda" if torch.cuda.is_available() else "cpu"

# load pretrained processor, tokenizer, and model
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = VisionEncoderDecoderModel.from_pretrained("Neleac/timesformer-gpt2-video-captioning").to(device)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [ ]:
video_captions = []
for video_path in tqdm_notebook(sorted_video_files[:20]):
    # load video
    container = av.open(video_path)

    # extract evenly spaced frames from video
    ###
    # seg_len = container.streams.video[0].frames
    # clip_len = model.config.encoder.num_frames
    # indices = set(np.linspace(0, seg_len, num=clip_len, endpoint=False).astype(np.int64))

    frames = []
    container.seek(0)
    for i, frame in enumerate(container.decode(video=0)):
        if i in indices:
            frames.append(frame.to_ndarray(format="rgb24"))

    # generate caption
    gen_kwargs = {
        "min_length": 0,
        "max_length": 100,
        "num_beams": 4,
    }
    pixel_values = image_processor(frames, return_tensors="pt").pixel_values.to(device)
    tokens = model.generate(pixel_values, **gen_kwargs)
    caption = tokenizer.batch_decode(tokens, skip_special_tokens=True)[0]
    video_captions.append(caption)

  0%|          | 0/20 [00:00<?, ?it/s]

RuntimeError: shape '[1, 5, 14, 8, 768]' is invalid for input of size 451584

In [ ]:
indices

[140, 281, 421]

In [ ]:
caption

'A man and a woman are playing a video game together.'

In [ ]:
for index in range(len(video_captions)):
    row = df.iloc[index]
    print(f'link:{row.link}')
    print(f'description:{row.description}')
    print(f'video_caption:{video_captions[index]}')
    print(20*'-')

link:https://cdn-st.rutubelist.ru/media/b0/e9/ef285e0241139fc611318ed33071/fhd.mp4
description:#нарезкистримов , #dota2 , #cs2 , #fifa23 , #minecraft , #майнкрафт , #геншин , #genshin
video_caption:A group of people are playing a game of soccer on a field.
--------------------
link:https://cdn-st.rutubelist.ru/media/39/6c/b31bc6864bef9d8a96814f1822ca/fhd.mp4
description:🤫НЕ ВВОДИ ЭТУ КОМАНДУ В РОБЛОКС ! #shorts #roblox #роблокс
video_caption:A man and a woman are standing in front of a wall and talking to each other.
--------------------
link:https://cdn-st.rutubelist.ru/media/e9/e0/b47a9df14a5e97942715e5e705c0/fhd.mp4
description:#boobs , #красивыедевушки , #ass
video_caption:A young woman and a young man are posing for a picture.
--------------------
link:https://cdn-st.rutubelist.ru/media/87/43/b11df3f344d0af773aac81e410ee/fhd.mp4
description:nan
video_caption:A woman is sitting at a table holding a card and talking about it.
--------------------
link:https://cdn-st.rutubelist.ru/me

### [movie-picture-captioning](https://huggingface.co/dumperize/movie-picture-captioning)

In [ ]:
from PIL import Image
import torch
from transformers import AutoTokenizer, ViTFeatureExtractor, VisionEncoderDecoderModel
import av

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dumperize/movie-picture-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("dumperize/movie-picture-captioning")
model = VisionEncoderDecoderModel.from_pretrained("dumperize/movie-picture-captioning")

tokenizer_config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/5.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

In [ ]:
max_length = 128
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
video_captions = []
for video_path in tqdm_notebook(sorted_video_files[:5]):
    # load video
    container = av.open(video_path)

    # get fps (corrected approach)
    fps = container.streams.video[0].average_rate.numerator / container.streams.video[0].average_rate.denominator

    # extract frames at 25%, 50%, & 75%
    seg_len = container.streams.video[0].frames  # total number of frames
    indices = [int(0.25 * seg_len), int(0.5 * seg_len), int(0.75 * seg_len)]
    frames = []

    container.seek(0)
    for i, frame in enumerate(container.decode(video=0)):
        if i in indices:
          frames.append(frame.to_ndarray(format="rgb24"))
    # generate caption
    gen_kwargs = {
        "min_length": 0,
        "max_length": 100,
        "num_beams": 4,
    }

    pixel_values = feature_extractor(images=frames, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    output_ids = model.generate(pixel_values, **gen_kwargs)

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    caption = '|'.join([pred.strip() for pred in preds])
    video_captions.append(caption)

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
for index in range(len(sorted_video_files[:5])):
    row = df.iloc[index]
    print(f'link:{row.link}')
    print(f'description:{row.description}')
    print(f'video_caption:{video_captions[index]}')
    print(20*'-')

link:https://cdn-st.rutubelist.ru/media/b0/e9/ef285e0241139fc611318ed33071/fhd.mp4
description:#нарезкистримов , #dota2 , #cs2 , #fifa23 , #minecraft , #майнкрафт , #геншин , #genshin
video_caption:В основе сюжета фильма лежит история тренерской команды средней школы в парнях которого пятнадцатилетней давности, а тренер и герой фильма ( в прошлом - первая женщина - тренер ) Кэти, в настоящее время одинокий и помешанный на всю жизнь тренер. Кэти мечтает о спортивной карьере, но в итоге фильма попадает в руки бывшего тренера, а он, в итоге, меняет полностью свою жизнь.|В основе сюжета фильма лежит история игроков, помешанных на становлении легендарного вида спорта спорта. После несчастного случая во время игры в названием « Скрэбдинг », в 1992 году команда решает принять участие в чемпионате мира по футболу.|Фильм - портрет о том, как мы воображаем и изображаем смерть. Герои фильма живут в разных странах мира и ( в том числе и в России ). У каждого из нас есть своя история и свои правила

### проба моделей, которые достают текст из скринов

### проба моделей audio2text [whishper hf link](https://huggingface.co/openai)

In [ ]:
# !pip install

In [ ]:
# !pip install moviepy

In [ ]:
from moviepy.editor import VideoFileClip
import whisper

In [ ]:
# Шаг 1: Извлечение аудио из видео и сохранение его в формате MP3
# Создаем директорию для сохранения аудио файлов
audio_output_dir = "audio_files"
os.makedirs(audio_output_dir, exist_ok=True)

for video_file in sorted_video_files[:10]:
    video_clip = VideoFileClip(video_file)
    audio_file_path = os.path.join(audio_output_dir, os.path.basename(video_file).replace(".mp4", ".mp3"))
    video_clip.audio.write_audiofile(audio_file_path)

chunk:  86%|████████████████████████████████████████████████████████████████████████████▉            | 351/406 [00:29<00:00, 1309.79it/s, now=None]

MoviePy - Writing audio in audio_files/0_b0_e9_ef285e0241139fc611318ed33071_fhd.mp3



chunk:  86%|████████████████████████████████████████████████████████████████████████████▉            | 351/406 [00:29<00:00, 1309.79it/s, now=None]

MoviePy - Done.


chunk:  86%|████████████████████████████████████████████████████████████████████████████▉            | 351/406 [00:29<00:00, 1309.79it/s, now=None]

...

MoviePy - Done.


chunk:  86%|████████████████████████████████████████████████████████████████████████████▉            | 351/406 [00:34<00:00, 1309.79it/s, now=None]

MoviePy - Writing audio in audio_files/9_fc_73_d86600d04e519178a55774dfe65e_fhd.mp3



chunk:  86%|████████████████████████████████████████████████████████████████████████████▉            | 351/406 [00:35<00:00, 1309.79it/s, now=None]

MoviePy - Done.


In [ ]:
# Загружаем модель Whisper
model = whisper.load_model("small")

# Транскрибация аудио файлов
audio_files = [os.path.join(audio_output_dir, f) for f in os.listdir(audio_output_dir) if f.endswith(".mp3")]

100%|███████████████████████████████████████| 461M/461M [01:07<00:00, 7.20MiB/s]


In [ ]:
sorted_audio_files = sorted(audio_files, key=lambda x: int(os.path.basename(x).split('_')[0]))

In [ ]:
transcriptions = []
results_audio = []
for audio_file in tqdm_notebook(sorted_audio_files[:10]):
    # Транскрибация аудио
    result = model.transcribe(audio_file, language='ru')
    results_audio.append(result)
    transcriptions.append(result["text"])

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
for index in range(len(sorted_audio_files[:10])):
    row = df.iloc[index]
    print(index)
    print(f'link:{row.link}')
    print(f'description:{row.description}')
    print(f'transcription:{transcriptions[index]}')
    print(20*'-')

0
link:https://cdn-st.rutubelist.ru/media/b0/e9/ef285e0241139fc611318ed33071/fhd.mp4
description:#нарезкистримов , #dota2 , #cs2 , #fifa23 , #minecraft , #майнкрафт , #геншин , #genshin
transcription: Субтитры мы SimilarlyStory
--------------------
1
link:https://cdn-st.rutubelist.ru/media/39/6c/b31bc6864bef9d8a96814f1822ca/fhd.mp4
description:🤫НЕ ВВОДИ ЭТУ КОМАНДУ В РОБЛОКС ! #shorts #roblox #роблокс
transcription: Сейчас я тебе покажу секретную команду в Roblox'е! Чтобы её активировать поставь лайк и подпишись, а также в виде в чат команду I love you! Когда вы её ведёте у вас на экране появится вот такой клоун! Скривц был создан одним из создателей Roblox'а и работает только в эракде есть Roblox Developer Service. Чтобы в Раскриме ранжа всего-то выйти из игры, но лучше не проверяй!
--------------------
2
link:https://cdn-st.rutubelist.ru/media/e9/e0/b47a9df14a5e97942715e5e705c0/fhd.mp4
description:#boobs , #красивыедевушки , #ass
transcription: Смотрите такие игрики в один раз.
-----

# Идеи полей

- синонимы для хэштего: можно взять совстречаемость тегов и попробовать распарсить какой-нибудь словарь синонимов
- Классификация языка аудио, чтобы потом asr нужного языка применять
- (+) транскрибация аудио в текст (audio to text) - вишпер
- (+) хэштеги по тексту - text classification, text summarization, chatgpt
- (+) шазам, если есть известная песня, выводим в хэштег ее название - shazamio
- транскрибация изображения, если есть текст (image to text)
- хэштеги по фото (по первому, среднему и последнему) - image classification / image captioning
- распознавание известных личностей по фото (милохин, мелстрой и тд)

In [ ]:
!pip install shazamio

  Using cached anyio-4.3.0-py3-none-any.whl.metadata (4.6 kB)
Using cached anyio-4.3.0-py3-none-any.whl (85 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.6/904.6 kB 1.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: anyio
    Found existing installation: anyio 4.2.0
    Uninstalling anyio-4.2.0:
      Successfully uninstalled anyio-4.2.0


In [ ]:
import asyncio
from shazamio import Shazam
import nest_asyncio
# Разрешаем повторный запуск цикла событий в Jupyter Notebook
nest_asyncio.apply()

async def recognize_audio(file_path):
    shazam = Shazam()
    out = await shazam.recognize_song(file_path)
    return out

In [ ]:
file_path = sorted_audio_files[7]
file_path

'audio_files/7_19_9c_98de3b4a4d9e83cb5dda828f899e_fhd.mp3'

In [ ]:
loop = asyncio.get_event_loop()
result = loop.run_until_complete(recognize_audio(file_path))

# Вывод информации о песне
if result and 'track' in result:
    track = result['track']
    print(f"Title: {track['title']}")
    print(f"Subtitle: {track['subtitle']}")
    print(f"URL: {track['url']}")
else:
    print("Song not recognized")

Title: What Was I Made For? (From The Motion Picture "Barbie")
Subtitle: Billie Eilish
URL: https://www.shazam.com/track/667426645/what-was-i-made-for-from-the-motion-picture-barbie
